In [89]:
import java.nio.file.Paths
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
//now create session
val spark: org.apache.spark.sql.SparkSession = SparkSession
      .builder()
      .appName("Amadeus challenge")
      .config("spark.master", "local")
      .getOrCreate()

//set new runtime options
spark.conf.set("spark.sql.shuffle.partitions", 2)
spark.conf.set("spark.executor.memory", "3g")
import spark.implicits._

2019-08-11 21:36:24 WARN  SparkSession$Builder:66 - Using an existing SparkSession; some configuration may not take effect.


import java.nio.file.Paths
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@1c667d8a
import spark.implicits._


In [90]:
import org.apache.spark.storage.StorageLevel._

import org.apache.spark.storage.StorageLevel._


## Read files

In [91]:
val root_path: String = "./";//where to find files
val searches_path: String = root_path + "big_searches.csv";
val bookings_path: String = root_path + "big_bookings.csv";

root_path: String = ./
searches_path: String = ./big_searches.csv
bookings_path: String = ./big_bookings.csv


In [92]:
val searches = sc.textFile(searches_path) // load RDD1
val bookings = sc.textFile(bookings_path) // load RDD2

searches: org.apache.spark.rdd.RDD[String] = ./big_searches.csv MapPartitionsRDD[48] at textFile at <console>:83
bookings: org.apache.spark.rdd.RDD[String] = ./big_bookings.csv MapPartitionsRDD[50] at textFile at <console>:84


In [93]:
val headerColumns_searches = searches.first().split("\\^").to[List]
val head_searches = sc.textFile(searches_path).first()


headerColumns_searches: List[String] = List("Date", "Time", "TxnCode", "OfficeID", "Country", "Origin", "Destination", "RoundTrip", "NbSegments", "Seg1Departure", "Seg1Arrival", "Seg1Date", "Seg1Carrier", "Seg1BookingCode", "Seg2Departure", "Seg2Arrival", "Seg2Date", "Seg2Carrier", "Seg2BookingCode", "Seg3Departure", "Seg3Arrival", "Seg3Date", "Seg3Carrier", "Seg3BookingCode", "Seg4Departure", "Seg4Arrival", "Seg4Date", "Seg4Carrier", "Seg4BookingCode", "Seg5Departure", "Seg5Arrival", "Seg5Date", "Seg5Carrier", "Seg5BookingCode", "Seg6Departure", "Seg6Arrival", "Seg6Date", "Seg6Carrier", "Seg6BookingCode", "From", "IsPublishedForNeg", "IsFromInternet", "IsFromVista", "TerminalID", "InternetOffice")
head_searches: String = "Date"^"Time"^"TxnCode"^"OfficeID"^"Country"^"Origin"^"Destinati...

In [94]:

val headerColumns_bookings = bookings.first().split("\\^").to[List]
val head_bookings = sc.textFile(bookings_path).first()

headerColumns_bookings: List[String] = List("act_date           ", "source", "pos_ctry", "pos_iata", "pos_oid  ", "rloc          ", "cre_date           ", "duration", "distance", "dep_port", "dep_city", "dep_ctry", "arr_port", "arr_city", "arr_ctry", "lst_port", "lst_city", "lst_ctry", "brd_port", "brd_city", "brd_ctry", "off_port", "off_city", "off_ctry", "mkt_port", "mkt_city", "mkt_ctry", "intl", "route          ", "carrier", "bkg_class", "cab_class", "brd_time           ", "off_time           ", "pax", "year", "month", "oid      ")
head_bookings: String = "act_date           "^"source"^"pos_ctry"^"pos_iata"^"pos_oid  "^"rloc          "^"cre_date           "^"duration"^"distance"^"dep_port"^"dep_city"^"dep_ctry"^"arr_port"^"arr_city"^"arr_ctry"^"lst_port"^"lst_city"^"lst_ctry"^"brd_...

In [95]:
head_bookings

res26: String = "act_date           "^"source"^"pos_ctry"^"pos_iata"^"pos_oid  "^"rloc          "^"cre_date           "^"duration"^"distance"^"dep_port"^"dep_city"^"dep_ctry"^"arr_port"^"arr_city"^"arr_ctry"^"lst_port"^"lst_city"^"lst_ctry"^"brd_port"^"brd_city"^"brd_ctry"^"off_port"^"off_city"^"off_ctry"^"mkt_port"^"mkt_city"^"mkt_ctry"^"intl"^"route          "^"carrier"^"bkg_class"^"cab_class"^"brd_time           "^"off_time           "^"pax"^"year"^"month"^"oid      "


In [96]:
val searches_count = searches.count()

searches_count: Long = 20008000


In [97]:
val bookings_count = bookings.count()

bookings_count: Long = 10009000


## Exercise 2
### Top 10 arrival airports in world in 2013

In [98]:
/* Lets define a function to take each line and return what we want
 * In particular, we want the column 12 with arr_port and 34 with pax
 * VIP: to handle the wrong lines reading (ex. 14 fields) we need to cover that exception, 
 * otherwise it would corrupt the RDD and give many headaches...
 */
def parseLineBookings(line:String) = {
    val fields   = line.split("\\^")
    //check if the line is complete..
    if (fields.length>34){
    val arr_port = fields(12).stripPrefix("\"").stripSuffix("\"").trim
    val paxstring = fields(34).stripPrefix("\"").stripSuffix("\"").trim
    if (arr_port=="" || paxstring=="") ("KKK","0") else
    (arr_port, paxstring)
    }
    else ("KKK","0")
}

parseLineBookings: (line: String)(String, String)


In [99]:
//Load the pair RDD from bookings with arr_port and pax
val pair_rdd = bookings.map(parseLineBookings).persist(MEMORY_AND_DISK)

pair_rdd: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[55] at map at <console>:87


In [100]:
pair_rdd.take(10)//test

res27: Array[(String, String)] = Array((arr_port,pax), (LHR,-1), (CLT,1), (CLT,1), (SVO,1), (SVO,1), (LGA,1), (LGA,1), (SIN,2), (SIN,2))


In [101]:
val kk = pair_rdd.filter(_._1 != "arr_port" ).persist().filter(_._1 == "KKK").collect()

kk: Array[(String, String)] = Array()


In [102]:
val pair_rdd_good = pair_rdd.filter(_._1 != "arr_port" ).filter(_._1 != "KKK").persist()//now create filtered pairRDD

pair_rdd_good: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[59] at filter at <console>:88


In [105]:
//Define a conversion to integer with exception handling. 
//This is important to avoid NaN when trying to convert your strings to integers
def toInt(s: String): Option[Int] = {
  try {
    Some(s.toInt)
  } catch {
    case e: Exception => None
  }
}

toInt: (s: String)Option[Int]


In [106]:
//We map with some care using match to handle the exceptions. If we find a None we will write 0 pax.
//Make sure wee have integers
val pair_bookings = pair_rdd_good.map(x=> {
      val tst = toInt(x._2)
      tst  match {
      case None => (x._1, 0)
      case Some(y) => (x._1, y) }
}).persist(MEMORY_AND_DISK)

pair_bookings: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[60] at map at <console>:94


In [107]:
pair_bookings.take(10)//test

res29: Array[(String, Int)] = Array((LHR,-1), (CLT,1), (CLT,1), (SVO,1), (SVO,1), (LGA,1), (LGA,1), (SIN,2), (SIN,2), (SIN,2))


In [108]:
pair_bookings.take(10)//test

res30: Array[(String, Int)] = Array((LHR,-1), (CLT,1), (CLT,1), (SVO,1), (SVO,1), (LGA,1), (LGA,1), (SIN,2), (SIN,2), (SIN,2))


In [109]:
//reduceByKey will groupByKey and then do the required operation on the values!
val temp = pair_bookings.reduceByKey(_+_)

temp: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[61] at reduceByKey at <console>:95


In [110]:
temp.take(5)//test

res31: Array[(String, Int)] = Array((DAY,2002), (HNL,34034), (BGI,10010), (IKT,0), (AJA,7007))


In [111]:
//In order to use the efficient transformation sortByKey we need to swap Keys and Values first.
//val top10 = temp.map(x => (x._2,x._1)).sortByKey(false).take(10)
val top10 = temp.map(_.swap).sortByKey(false).take(10)//swap does the trick

top10: Array[(Int, String)] = Array((112112,HKG), (95095,LGA), (94094,ORD), (92092,JFK), (91091,LAX), (91091,SFO), (90090,MCO), (82082,DCA), (79079,DEN), (76076,LHR))


In [112]:
top10.foreach(e => println(e))

(112112,HKG)
(95095,LGA)
(94094,ORD)
(92092,JFK)
(91091,LAX)
(91091,SFO)
(90090,MCO)
(82082,DCA)
(79079,DEN)
(76076,LHR)
